# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770799610797                   -0.53    9.0    179ms
  2   -2.772138906124       -2.87       -1.30    1.0    108ms
  3   -2.772170201544       -4.50       -2.64    1.0    112ms
  4   -2.772170697530       -6.30       -4.01    1.0    125ms
  5   -2.772170722869       -7.60       -4.47    2.0    138ms
  6   -2.772170723007       -9.86       -5.53    1.0    546ms
  7   -2.772170723015      -11.11       -5.71    2.0    126ms
  8   -2.772170723015      -12.41       -6.18    1.0    127ms
  9   -2.772170723015      -13.28       -6.87    1.0    140ms
 10   -2.772170723015      -14.03       -7.15    2.0    131ms
 11   -2.772170723015   +  -14.45       -8.51    1.0    120ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770788499961                   -0.52    9.0    182ms
  2   -2.7

1.7735580511504394

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770767515414                   -0.53    9.0    195ms
  2   -2.772056707738       -2.89       -1.31    1.0    123ms
  3   -2.772083126098       -4.58       -2.59    1.0    111ms
  4   -2.772083380949       -6.59       -3.50    1.0    116ms
  5   -2.772083415949       -7.46       -3.90    2.0    134ms
  6   -2.772083417772       -8.74       -5.30    1.0    563ms
  7   -2.772083417810      -10.42       -5.57    2.0    129ms
  8   -2.772083417811      -12.34       -6.12    1.0    114ms
  9   -2.772083417811      -13.20       -6.33    2.0    141ms
 10   -2.772083417811      -13.45       -7.99    1.0    120ms
 11   -2.772083417811   +  -13.81       -8.15    2.0    135ms

Polarizability via ForwardDiff:       1.7725349604603038
Polarizability via finite difference: 1.7735580511504394
